<a href="https://colab.research.google.com/github/thwlswkdgh/GAN/blob/main/code_practice/SIMGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 환경설정
1. kaggle 접속
2. 데이터 다운로드 및 zip파일 풀기

In [1]:
!pip install -q kaggle

In [2]:
!mkdir -p ~/.kaggle

In [3]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [4]:
!cp kaggle.json ~/.kaggle/

In [5]:
!chmod 600 /root/.kaggle/kaggle.json

In [6]:
!kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              237KB  2021-12-12 11:59:54          18828  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01          11425  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           5875  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   2GB  2021-10-22 10:48:21           4008  
promptcloud/careerbuilder-job-listing-2020                  Careerbuilder Jo

In [7]:
!kaggle datasets download -d 4quant/eye-gaze -p /data/

100% 4.45G/4.46G [01:21<00:00, 91.1MB/s]
100% 4.46G/4.46G [01:22<00:00, 58.4MB/s]


In [ ]:
!unzip /data/eye-gaze.zip -d eye-gaze

In [35]:
!git clone https://github.com/PacktPublishing/Generative-Adversarial-Networks-Cookbook.git
!mkdir out

Cloning into 'Generative-Adversarial-Networks-Cookbook'...
remote: Enumerating objects: 177, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 177 (delta 1), reused 0 (delta 0), pack-reused 172
Receiving objects: 100% (177/177), 94.49 KiB | 3.05 MiB/s, done.
Resolving deltas: 100% (51/51), done.


### 손실함수
1. self regulation loss
2. local adversarial loss

In [69]:
#!/usr/bin/env python3
import tensorflow as tf


def self_regularization_loss(y_true,y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    return tf.multiply(0.0002,tf.reduce_sum(tf.abs(y_pred-y_true)))

def local_adversarial_loss(y_true,y_pred):
    truth = tf.reshape(y_true,(-1,2))
    predicted = tf.reshape(y_pred,(-1,2))

    computed_loss = tf.nn.softmax_cross_entropy_with_logits(labels=truth,logits=predicted)
    output = tf.reduce_mean(computed_loss)
    return output


### 판별기

In [70]:
#!/usr/bin/env python3
import sys
import numpy as np
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization, Lambda, Concatenate, MaxPooling2D
from keras.layers.core import Activation
from keras.layers.convolutional import Convolution2D
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import relu
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD,Nadam, Adamax
from tensorflow.keras.utils import plot_model
import tensorflow as tf



class Discriminator(object):
    def __init__(self, width = 35, height= 55, channels = 1,name='discriminator'):
        self.W = width
        self.H = height
        self.C = channels
        self.SHAPE = (height,width,channels)
        self.NAME = name
        
        self.Discriminator = self.model()
        self.OPTIMIZER = SGD(learning_rate=0.001)
        self.Discriminator.compile(loss=local_adversarial_loss, optimizer=self.OPTIMIZER)

        self.save_model_graph()
        self.summary()
#model.add(Convolution2D(64, kernel_size= 5, strides= 2, input_shape=(self.W,self.H,self.C), padding='same',activation=LeakyReLU(alpha=0.2)))
    def model(self):
        input_layer = Input(shape=self.SHAPE)
        x = Convolution2D(96,kernel_size= 3, strides =2, padding='same',activation='relu')(input_layer)
        x = Convolution2D(64,kernel_size= 3, strides =2, padding='same',activation='relu')(x)
        x = MaxPooling2D(pool_size=(3,3),padding='same')(x)
        x = Convolution2D(32,kernel_size= 3, strides =1, padding='same',activation='relu')(x)
        x = Convolution2D(32,kernel_size= 1, strides =1, padding='same',activation='relu')(x)
        x = Convolution2D(2,kernel_size= 1, strides =1, padding='same',activation='relu')(x)
        output_layer = Reshape((-1,2))(x)
        return Model(input_layer,output_layer) 

    def summary(self):
        return self.Discriminator.summary()

    def save_model_graph(self):
        plot_model(self.Discriminator, to_file='/data/Discriminator_Model.png')

    def save_model(self,epoch,batch):
        self.Discriminator.save('/data/'+self.NAME+'_Epoch_'+epoch+'_Batch_'+batch+'model.h5')
d = Discriminator()

Model: "model_52"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_69 (InputLayer)       [(None, 55, 35, 1)]       0         
                                                                 
 conv2d_297 (Conv2D)         (None, 28, 18, 96)        960       
                                                                 
 conv2d_298 (Conv2D)         (None, 14, 9, 64)         55360     
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 5, 3, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_299 (Conv2D)         (None, 5, 3, 32)          18464     
                                                                 
 conv2d_300 (Conv2D)         (None, 5, 3, 32)          1056      
                                                          

###생성기

In [71]:
import sys
import numpy as np
from keras.layers import Dense, Reshape, Input, BatchNormalization, Concatenate, Activation
from keras.layers.core import Activation
from keras.layers.convolutional import UpSampling2D, Convolution2D, MaxPooling2D,Deconvolution2D, Conv2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, Nadam,Adamax
from keras import initializers
from keras import layers
import tensorflow as tf


class Generator(object):
    def __init__(self, width = 35, height= 55, channels = 1,name='generator'):
        
        self.W = width
        self.H = height
        self.C = channels
        self.SHAPE = (height,width,channels)
        self.NAME = name

        self.Generator = self.model()
        self.OPTIMIZER = SGD(learning_rate=0.001)
        self.Generator.compile(loss=self_regularization_loss, optimizer=self.OPTIMIZER)

        self.save_model_graph()
        self.summary()

    def model(self):
        # Input
        input_layer = Input(shape=self.SHAPE)
        x = Convolution2D(64, kernel_size= 3, padding='same',activation='relu')(input_layer)

        # ResNet Block 1
        res_x_input_1 = Conv2D(64, kernel_size= 3, padding='same',activation='relu')(x)
        x = Convolution2D(64, kernel_size= 3, padding='same',activation='relu')(res_x_input_1)
        x = layers.Add()([res_x_input_1,x])
        x = Activation('relu')(x)

        # ResNet Block 2
        res_x_input_2 = Conv2D(64, kernel_size= 3, padding='same',activation='relu')(x)
        x = Convolution2D(64, kernel_size= 3, padding='same',activation='relu')(res_x_input_2)
        x = layers.Add()([res_x_input_2,x])
        x = Activation('relu')(x)

        # ResNet Block 3
        res_x_input_3 = Conv2D(64, kernel_size= 3, padding='same',activation='relu')(x)
        x = Convolution2D(64,kernel_size= 3, padding='same',activation='relu')(res_x_input_3)
        x = layers.Add()([res_x_input_3,x])
        x = Activation('relu')(x)

        # ResNet Block 4
        res_x_input_4 = Conv2D(64, kernel_size= 3, padding='same',activation='relu')(x)
        x = Convolution2D(64,kernel_size= 3, padding='same',activation='relu')(res_x_input_4)
        x = layers.Add()([res_x_input_4,x])
        x = Activation('relu')(x)

        output_layer = Convolution2D(self.C,kernel_size= 1, padding='same',activation='tanh')(x)
        
        return Model(input_layer,output_layer)

    def summary(self):
        return self.Generator.summary()

    def save_model_graph(self):
        plot_model(self.Generator, to_file='/data/Generator_Model.png')
    
    def save_model(self,epoch,batch):
        self.Generator.save('/data/'+self.NAME+'_Epoch_'+epoch+'_Batch_'+batch+'model.h5')
g=Generator()

Model: "model_53"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_70 (InputLayer)          [(None, 55, 35, 1)]  0           []                               
                                                                                                  
 conv2d_302 (Conv2D)            (None, 55, 35, 64)   640         ['input_70[0][0]']               
                                                                                                  
 conv2d_303 (Conv2D)            (None, 55, 35, 64)   36928       ['conv2d_302[0][0]']             
                                                                                                  
 conv2d_304 (Conv2D)            (None, 55, 35, 64)   36928       ['conv2d_303[0][0]']             
                                                                                           

### GAN


In [72]:
#!/usr/bin/env python3
import sys
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Input
import tensorflow as tf


class GAN(object):
    def __init__(self, model_inputs=[],model_outputs=[], name='gan'):
        self.OPTIMIZER = SGD(learning_rate=2e-4,nesterov=True)
        self.NAME=name

        self.inputs = model_inputs
        self.outputs = model_outputs
        self.gan_model = Model(inputs = self.inputs, outputs = self.outputs)
        self.OPTIMIZER = SGD(learning_rate=0.001)
        self.gan_model.compile(loss=[self_regularization_loss, self_regularization_loss],
                            optimizer=self.OPTIMIZER)
        self.save_model_graph()
        self.summary()

    def summary(self):
        return self.gan_model.summary()

    def save_model_graph(self):
        plot_model(self.gan_model, to_file='/data/GAN_Model.png')

    def save_model(self,epoch,batch):
        self.gan_model.save('/data/'+self.NAME+'_Epoch_'+epoch+'_Batch_'+batch+'model.h5')

    def self_regularization_loss(self,y_true,y_pred):
        return tf.multiply(0.0002,tf.reduce_sum(tf.abs(y_pred-y_true)))

    def local_adversarial_loss(self,y_true,y_pred):
        truth = tf.reshape(y_true,(-1,2))
        predicted = tf.reshape(y_pred,(-1,2))

        computed_loss = tf.nn.softmax_cross_entropy_with_logits(labels=truth,logits=predicted)
        output = tf.reduce_mean(computed_loss)
        return output

###Train

In [77]:
#!/usr/bin/env python3
from keras.datasets import mnist
from keras.layers import Input
from random import randint
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import numpy as np
from copy import deepcopy

class Trainer:
    def __init__(self, height=55,width=35, channels=1,epochs =100, batch=16, checkpoint=50,sim_path='',real_path='',data_limit=0.001,generator_steps=2,discriminator_steps=1):
        self.W = width
        self.H = height
        self.C = channels
        self.EPOCHS = epochs
        self.BATCH = batch
        self.CHECKPOINT = checkpoint
        self.DATA_LIMIT=data_limit
        self.GEN_STEPS = generator_steps
        self.DISC_STEPS = discriminator_steps

        self.X_real = self.load_h5py(real_path)
        self.X_sim = self.load_h5py(sim_path)

        self.refiner = Generator(height=self.H, width=self.W, channels=self.C)
        self.discriminator = Discriminator(height=self.H, width=self.W, channels=self.C)
        self.discriminator.trainable = False

        self.synthetic_image = Input(shape=(self.H, self.W, self.C))
        self.real_or_fake = Input(shape=(self.H, self.W, self.C))


        self.refined_image = self.refiner.Generator(self.synthetic_image)
        self.discriminator_output = self.discriminator.Discriminator(self.real_or_fake)
        self.combined = self.discriminator.Discriminator(self.refined_image)

        model_inputs  = [self.synthetic_image]
        model_outputs = [self.refined_image, self.combined]
        self.gan = GAN(model_inputs=model_inputs,model_outputs=model_outputs)

    def load_h5py(self,data_path):
        with h5py.File(data_path,'r') as t_file:
            print('Images found:',len(t_file['image']))
            image_stack = np.stack([np.expand_dims(a,-1) for a in t_file['image'].values()],0)
        return image_stack

    def train(self):
        for e in range(self.EPOCHS):

            b = 0
            X_real_temp = deepcopy(self.X_real)
            X_sim_temp = deepcopy(self.X_sim)
            combined_loss = np.zeros(shape=len(self.gan.gan_model.metrics_names))
            discriminator_loss_real = np.zeros(shape=len(self.discriminator.Discriminator.metrics_names))
            discriminator_loss_sim = np.zeros(shape=len(self.discriminator.Discriminator.metrics_names))

            while b < self.BATCH:
                # Keep track of Batches
                b=b+1

                count_real_images = int(self.BATCH)
                starting_indexs = randint(0, (min(len(X_real_temp),len(X_sim_temp))-count_real_images))
              
                real_images_raw = X_real_temp[ starting_indexs : (starting_indexs + count_real_images) ]
                real_images = real_images_raw.reshape( count_real_images, self.H, self.W, self.C )

                y_real = np.array([[[1.0, 0.0]] * self.discriminator.Discriminator.output_shape[1]] * self.BATCH)
                
                sim_images_raw = X_sim_temp[ starting_indexs : (starting_indexs + count_real_images) ]
                sim_images = sim_images_raw.reshape( count_real_images, self.H, self.W, self.C )

                y_sim = np.array([[[0.0, 1.0]] * self.discriminator.Discriminator.output_shape[1]] * self.BATCH)
                gen_step_first = True
                for _ in range(self.GEN_STEPS):
                  if gen_step_first==True:
                    gen_step_first=False
                    combined_loss = self.gan.gan_model.train_on_batch(sim_images,[sim_images, y_real])
                  else:
                    combined_loss = np.add(self.gan.gan_model.train_on_batch(sim_images,[sim_images, y_real]), combined_loss)
        
                dis_step_first = True
                for _ in range(self.DISC_STEPS):
                  improved_image_batch = self.refiner.Generator.predict_on_batch(sim_images)
                  if dis_step_first==True:
                    dis_step_first = False
                    discriminator_loss_real = self.discriminator.Discriminator.train_on_batch(real_images, y_real)
                    discriminator_loss_sim = self.discriminator.Discriminator.train_on_batch(improved_image_batch, y_sim)
                  else:
                    discriminator_loss_real = np.add(self.discriminator.Discriminator.train_on_batch(real_images, y_real), discriminator_loss_real)
                    discriminator_loss_sim = np.add(self.discriminator.Discriminator.train_on_batch(improved_image_batch, y_sim),discriminator_loss_sim)

                print ('Epoch: '+str(int(e))+', [Real Discriminator :: Loss: '+str(discriminator_loss_real)+'], [ GAN :: Loss: '+str(combined_loss)+']')
                        
        return

###Run

In [ ]:
#!/usr/bin/env python3
import h5py

# Command Line Argument Method
HEIGHT  = 55
WIDTH   = 35
CHANNELS = 1
EPOCHS = 100
BATCH = 16
CHECKPOINT = 50
SIM_PATH = "/content/eye-gaze/gaze.h5"
REAL_PATH = "/content/eye-gaze/real_gaze.h5"

trainer = Trainer(height=HEIGHT,width=WIDTH, channels=CHANNELS,epochs =EPOCHS,\
                 batch=BATCH,\
                 checkpoint=CHECKPOINT,\
                 sim_path=SIM_PATH,\
                 real_path=REAL_PATH)
trainer.train()

Images found: 34100
Images found: 50000
Model: "model_60"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_79 (InputLayer)          [(None, 55, 35, 1)]  0           []                               
                                                                                                  
 conv2d_342 (Conv2D)            (None, 55, 35, 64)   640         ['input_79[0][0]']               
                                                                                                  
 conv2d_343 (Conv2D)            (None, 55, 35, 64)   36928       ['conv2d_342[0][0]']             
                                                                                                  
 conv2d_344 (Conv2D)            (None, 55, 35, 64)   36928       ['conv2d_343[0][0]']             
                                                   